In [ ]:
from datetime import datetime, timedelta, timezone
import requests
import logging
import urllib3
import sys
import os
sys.path.append(os.path.abspath('../vault'))
from vault_methods import VaultMethods

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p',
                    level=logging.INFO)

vault_session = VaultMethods()
vault_session.authenticate()
gravwell_token = vault_session.retrieve_gravwell_secret()

if gravwell_token:
    logging.info("Retrieved SIEM token")
else:
    logging.error("Failed to retrieve SIEM token")

headers = {
             "Gravwell-Token": gravwell_token, 
             "Content-Type": "application/json"
          }

def convert_datetime(datetime_str):
    dt = datetime.fromisoformat(datetime_str)
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

def execute_direct_query(headers, siem: str, query: str) -> None:
    timestamp = datetime.now(timezone.utc) - timedelta(minutes=600)
    search_start = timestamp.isoformat()
    search_start = convert_datetime(search_start)
    search_conclude = datetime.now(timezone.utc).isoformat()
    search_start = convert_datetime(search_start)
    search_conclude = convert_datetime(search_conclude)
    payload = {"Background": True,
               "Name": "Direct Query",
               "NoHistory": True,
               "Preview": False,
               "Format": "text",
               "SearchEnd": search_conclude,
               "SearchStart": search_start,
               "SearchString": query}
    try:
        response = requests.post(headers=headers,
                                 url=f"https://{siem}/api/search/direct",
                                 json=payload,
                                 verify=False)
        response_content = response.content
        response_content = response_content.decode('utf-8')
        logging.info(response_content)
    except Exception as e:
        logging.exception("Error executing direct query: {0}".format(e))

siem = input("Enter the SIEM FQDN -> ")
query = "tag=_alerts"
execute_direct_query(headers, siem, query)